In [35]:
import os
import torch
import tarfile
import torchvision
import torch.nn as nn
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_url

In [36]:
transform_train = transforms.Compose([

    transforms.Resize((150, 150)),  #becasue vgg takes 150*150
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))

])

#Augmentation is not done for test/validation data.
transform_test = transforms.Compose([

    transforms.Resize((150, 150)),  #becasue vgg takes 150*150
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))

])

In [37]:
train_ds = ImageFolder("dataset/train/", transform=transform_train)
test_ds = ImageFolder("dataset/val/", transform=transform_test)
pred_ds = ImageFolder("dataset/pred/", transform=transform_test)

In [38]:
len(train_ds), len(test_ds), len(pred_ds)

(6341, 1450, 1450)

In [39]:
batch_size = 128
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(test_ds, batch_size, num_workers=4, pin_memory=True)
pred_dl = DataLoader(pred_ds, batch_size, num_workers=4, pin_memory=True)

In [41]:
len(pred_ds.classes)

145

In [42]:
import pathlib

root = pathlib.Path('dataset/train')
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes

['absa',
 'adidas',
 'adobe',
 'airbnb',
 'alibaba',
 'aliexpress',
 'allegro',
 'amazon',
 'ameli_fr',
 'american_express',
 'anadolubank',
 'aol',
 'apple',
 'arnet_tech',
 'aruba',
 'att',
 'azul',
 'bahia',
 'banco_de_occidente',
 'banco_inter',
 'bankia',
 'barclaycard',
 'barclays',
 'bbt',
 'bcp',
 'bestchange',
 'blizzard',
 'bmo',
 'bnp_paribas',
 'bnz',
 'boa',
 'bradesco',
 'bt',
 'caixa_bank',
 'canada',
 'capital_one',
 'capitec',
 'cathay_bank',
 'cetelem',
 'chase',
 'cibc',
 'cloudconvert',
 'cloudns',
 'cogeco',
 'commonwealth_bank',
 'cox',
 'crate_and_barrel',
 'cryptobridge',
 'daum',
 'db',
 'dhl',
 'dkb',
 'docmagic',
 'dropbox',
 'ebay',
 'eharmony',
 'erste',
 'etisalat',
 'etrade',
 'facebook',
 'fibank',
 'file_transfer',
 'fnac',
 'fsnb',
 'godaddy',
 'google',
 'google_drive',
 'gov_uk',
 'grupo_bancolombia',
 'hfe',
 'hsbc',
 'htb',
 'ics',
 'ieee',
 'impots_gov',
 'infinisource',
 'instagram',
 'irs',
 'itau',
 'knab',
 'la_banque_postale',
 'la_poste',
 '

In [43]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [44]:
from torchvision import models

vgg16 = models.vgg16(pretrained=True)
resnet50 = models.resnet50(pretrained=True)

In [45]:
for p in vgg16.parameters():
    p.requires_grad = False

for p in resnet50.parameters():
    p.requires_grad = False

In [46]:
from torch import nn

classes_num = len(classes)
vgg16.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=2048),
    nn.ReLU(),
    nn.Linear(in_features=2048, out_features=512),
    nn.ReLU(),
    nn.Dropout(0.6),
    nn.Linear(in_features=512, out_features=classes_num),
    nn.LogSoftmax(dim=1)
)
resnet50.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024),
    nn.ReLU(),
    nn.Linear(in_features=1024, out_features=512),
    nn.ReLU(),
    nn.Dropout(p=0.6),
    nn.Linear(in_features=512, out_features=classes_num),
    nn.LogSoftmax(dim=1)
)

In [47]:
import torch
import torch.nn.functional as F


class ImageClassificationBase(nn.Module):

    def training_step(self, batch):
        images, labels = batch

        out = self(images)  # Generate predictions

        loss = F.cross_entropy(out, labels)  # Calculate loss

        return loss

    def validation_step(self, batch):
        images, labels = batch

        out = self(images)  # Generate predictions

        loss = F.cross_entropy(out, labels)  # Calculate loss

        acc = accuracy(out, labels)  # Calculate accuracy

        return {'val_loss': loss.detach(), 'val_acc': acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]

        epoch_loss = torch.stack(batch_losses).mean()  # Combine losses

        batch_accs = [x['val_acc'] for x in outputs]

        epoch_acc = torch.stack(batch_accs).mean()  # Combine accuracies

        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(

            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [48]:
class WebVGGModel(ImageClassificationBase):
    def __init__(self, model):
        super().__init__()
        self.network = model

    def forward(self, xb):
        return self.network(xb)


class WebResNetModel(ImageClassificationBase):
    def __init__(self, model):
        super().__init__()
        self.network = model

    def forward(self, xb):
        return self.network(xb)

In [49]:
model_vgg = WebVGGModel(vgg16)
model_resnet = WebResNetModel(resnet50)

In [50]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")


device = get_default_device()

In [51]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [52]:
class DeviceDataLoader:
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [53]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
pred_dl = DeviceDataLoader(pred_dl, device)
to_device(model_vgg, device);
to_device(model_resnet, device);

In [54]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        model.train()  #eval() is called to tell model that now it is training mode and so  perform stuff like dropout,backpropagation etc..
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [55]:
model_vgg = to_device(model_vgg, device)
evaluate(model_vgg, val_dl)

{'val_loss': 4.975980758666992, 'val_acc': 0.012369791977107525}

In [56]:
model_resnet = to_device(model_resnet, device)
evaluate(model_resnet, val_dl)

{'val_loss': 4.979844570159912, 'val_acc': 0.0026041667442768812}

In [57]:
num_epochs = 40
opt_func = torch.optim.Adam
lr = 0.00001
vgg_history = fit(num_epochs, lr, model_vgg, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 4.8486, val_loss: 4.9453, val_acc: 0.0280
Epoch [1], train_loss: 4.5876, val_loss: 4.8884, val_acc: 0.0495
Epoch [2], train_loss: 4.3422, val_loss: 4.7842, val_acc: 0.0794
Epoch [3], train_loss: 4.1346, val_loss: 4.6893, val_acc: 0.0931
Epoch [4], train_loss: 3.9587, val_loss: 4.6018, val_acc: 0.1016
Epoch [5], train_loss: 3.8096, val_loss: 4.5254, val_acc: 0.1035
Epoch [6], train_loss: 3.6592, val_loss: 4.4394, val_acc: 0.1237
Epoch [7], train_loss: 3.5345, val_loss: 4.4134, val_acc: 0.1257
Epoch [8], train_loss: 3.4378, val_loss: 4.3235, val_acc: 0.1406
Epoch [9], train_loss: 3.3466, val_loss: 4.2348, val_acc: 0.1582
Epoch [10], train_loss: 3.2533, val_loss: 4.1972, val_acc: 0.1628
Epoch [11], train_loss: 3.1664, val_loss: 4.1589, val_acc: 0.1725
Epoch [12], train_loss: 3.0969, val_loss: 4.0882, val_acc: 0.1869
Epoch [13], train_loss: 3.0283, val_loss: 4.0491, val_acc: 0.1940
Epoch [14], train_loss: 2.9451, val_loss: 3.9912, val_acc: 0.2005
Epoch [15], train_lo

In [58]:
num_epochs = 5
opt_func = torch.optim.Adam
lr = 0.0001
resnet_history = fit(num_epochs, lr, model_resnet, train_dl, val_dl, opt_func)


Epoch [0], train_loss: 4.7981, val_loss: 5.0822, val_acc: 0.0273
Epoch [1], train_loss: 4.5468, val_loss: 5.0009, val_acc: 0.0397
Epoch [2], train_loss: 4.3064, val_loss: 4.9273, val_acc: 0.0586
Epoch [3], train_loss: 4.0280, val_loss: 4.7698, val_acc: 0.0768
Epoch [4], train_loss: 3.7858, val_loss: 4.6384, val_acc: 0.1055


In [59]:
def predict_single(input, label, model):
    input = to_device(input, device)
    inputs = input.unsqueeze(0)  # unsqueeze the input i.e. add an additonal dimension
    predictions = model(inputs)
    prediction = predictions[0].detach().cpu()
    # print(f"Prediction is {np.argmax(prediction)} of Model whereas given label is {label}")
    return np.argmax(prediction)

In [60]:
from sklearn.metrics import classification_report

models = [model_vgg, model_resnet]
for modell in models:
    labels = []
    predicts = []
    for i, img in enumerate(pred_ds):
        result = predict_single(img[0], img[1], modell)
        labels.append(img[1])
        predicts.append(result)
    print("分类报告")
    report = classification_report(labels,predicts,zero_division=0)
    print(report)
    print("\n\n")

分类报告
              precision    recall  f1-score   support

           0       0.50      0.80      0.62        10
           1       0.50      0.70      0.58        10
           2       0.06      0.10      0.08        10
           3       0.50      1.00      0.67        10
           4       0.47      0.70      0.56        10
           5       0.62      0.50      0.56        10
           6       0.00      0.00      0.00        10
           7       0.50      0.10      0.17        10
           8       0.00      0.00      0.00        10
           9       0.25      0.20      0.22        10
          10       0.89      0.80      0.84        10
          11       0.00      0.00      0.00        10
          12       0.70      0.70      0.70        10
          13       0.46      0.60      0.52        10
          14       1.00      0.30      0.46        10
          15       0.11      0.60      0.19        10
          16       1.00      0.60      0.75        10
          17       1.0

In [41]:
torch.save(model_vgg.state_dict(), 'web_model_vgg.pth')
torch.save(model_resnet.state_dict(), 'web_model_resnet.pth')

In [42]:
from core import model

net = model.attention_net(topN=3)
net

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/mi/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100.0%


attention_net(
  (pretrained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequent

In [44]:
len("ABC abc")

7